In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn
from sklearn import model_selection as ms
from sklearn.linear_model import LogisticRegression

# sns.set(rc={'figure.figsize':(11.7,8.27)})

import gzip

import copy

import import_ipynb
from My_Functions import null_cols, rmse

importing Jupyter notebook from My_Functions.ipynb


In [2]:
acc = pd.read_csv('../../accepted_2007_to_2018Q4.csv.gz',
                  compression='gzip',
                  header=0,
                  sep=',',
                  quotechar='"')
acc

C:\Users\tdcho\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.00,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.00,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN

In [3]:
# we will remove those rows
rows_to_drop_id = list(acc[acc['id'].str.contains('Total|Loans', na=False)].index)

acc = acc.drop(rows_to_drop_id)
acc

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.00,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.00,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN

# Data Cleaning and Imputation

## Getting rows that contain the desired target

In [4]:
# keeping only the rows that have loan status of Paid Off, Default, or Charged Off

acc_train = copy.deepcopy(acc[(acc['loan_status'].str.contains('Paid|Default|Charged', na=False))])
acc_train = acc_train.reset_index()
acc_train = acc_train.drop('index', axis=1)
acc_train

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,N

In [5]:
# create target label

target_train = copy.deepcopy(acc_train[['loan_status']])

target_train

,loan_status
0,Fully Paid
1,Fully Paid
2,Fully Paid
3,Fully Paid
4,Fully Paid
...,...
1348094,Fully Paid
1348095,Fully Paid
1348096,Charged Off
1348097,Fully Paid


In [6]:
target_train[['loan_status']].value_counts()

loan_status                                        
Fully Paid                                             1076751
Charged Off                                             268559
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
dtype: int64

In [7]:
# converting loan statuses to binary (0, 1)

target_train['loan_status_binary'] = 1

# create mask for clarity on what is being changed
mask_fully = target_train['loan_status'].str.contains('Fully', na=False)

target_train.loc[mask_fully, 'loan_status_binary'] = 0

target_train = target_train.drop('loan_status', axis=1)

target_train

,loan_status_binary
0,0
1,0
2,0
3,0
4,0
...,...
1348094,0
1348095,0
1348096,1
1348097,0


In [8]:
target_train[['loan_status_binary']].value_counts()

loan_status_binary
0                     1078739
1                      269360
dtype: int64

In [9]:
acc_train = acc_train.drop(['loan_status'], axis=1)

acc_train

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,N

## Splitting the dataset into train and prediction

In [10]:
# getting indices of half of the samples

target_nodwnsmpl_index_0 = sorted(list(target_train\
                                       [target_train\
                                        ['loan_status_binary']==0].sample(frac=0.5,
                                                                          random_state=0).index))

target_nodwnsmpl_index_1 = sorted(list(target_train\
                                       [target_train\
                                        ['loan_status_binary']==1].sample(frac=0.5,
                                                                          random_state=0).index))

target_nodwnsmpl_index_0and1 = target_nodwnsmpl_index_0 + target_nodwnsmpl_index_1
target_nodwnsmpl_index_0and1

[0,
 5,
 6,
 7,
 8,
 14,
 15,
 19,
 20,
 22,
 25,
 28,
 32,
 35,
 37,
 38,
 42,
 43,
 53,
 56,
 57,
 58,
 61,
 64,
 65,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 77,
 78,
 82,
 85,
 87,
 90,
 94,
 96,
 97,
 98,
 99,
 102,
 104,
 105,
 109,
 110,
 113,
 115,
 116,
 119,
 123,
 126,
 129,
 133,
 135,
 142,
 144,
 153,
 155,
 156,
 162,
 165,
 167,
 171,
 172,
 173,
 175,
 176,
 179,
 180,
 183,
 184,
 185,
 186,
 190,
 191,
 193,
 195,
 199,
 200,
 203,
 204,
 205,
 206,
 207,
 209,
 215,
 216,
 218,
 220,
 222,
 225,
 226,
 228,
 229,
 230,
 231,
 234,
 235,
 238,
 243,
 249,
 251,
 255,
 258,
 261,
 262,
 265,
 266,
 267,
 271,
 273,
 278,
 279,
 283,
 284,
 285,
 295,
 298,
 299,
 301,
 302,
 304,
 309,
 311,
 312,
 314,
 322,
 323,
 326,
 328,
 329,
 330,
 332,
 343,
 344,
 347,
 348,
 349,
 353,
 356,
 358,
 359,
 360,
 361,
 363,
 366,
 367,
 368,
 371,
 374,
 380,
 381,
 385,
 386,
 388,
 390,
 391,
 395,
 396,
 397,
 400,
 402,
 403,
 407,
 411,
 413,
 416,
 420,
 422,
 423,
 426,
 427

In [11]:
len(target_nodwnsmpl_index_0and1)

674050

In [12]:
# getting the other half of the indices

target_nodwnsmpl_pred_index = sorted(list(set(list(range(0,
                                               target_train['loan_status_binary'].count()))) -\
                                set(target_nodwnsmpl_index_0and1)))

target_nodwnsmpl_pred_index

[1,
 2,
 3,
 4,
 9,
 11,
 12,
 13,
 16,
 17,
 18,
 21,
 23,
 24,
 26,
 27,
 29,
 30,
 31,
 33,
 34,
 39,
 40,
 41,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 55,
 59,
 60,
 62,
 63,
 66,
 76,
 79,
 80,
 81,
 84,
 86,
 89,
 91,
 92,
 93,
 95,
 101,
 103,
 106,
 108,
 112,
 114,
 117,
 118,
 120,
 122,
 124,
 125,
 128,
 131,
 132,
 134,
 137,
 138,
 139,
 140,
 141,
 143,
 145,
 146,
 147,
 148,
 149,
 150,
 152,
 157,
 158,
 159,
 161,
 163,
 164,
 166,
 168,
 169,
 170,
 174,
 177,
 181,
 182,
 187,
 189,
 192,
 194,
 196,
 197,
 198,
 201,
 208,
 210,
 211,
 212,
 214,
 217,
 219,
 221,
 223,
 224,
 227,
 232,
 233,
 236,
 237,
 239,
 241,
 242,
 244,
 245,
 246,
 247,
 248,
 250,
 252,
 253,
 254,
 256,
 257,
 260,
 263,
 264,
 269,
 270,
 272,
 274,
 275,
 276,
 277,
 282,
 286,
 287,
 288,
 289,
 291,
 292,
 293,
 294,
 296,
 297,
 303,
 306,
 307,
 308,
 310,
 313,
 315,
 317,
 318,
 319,
 321,
 324,
 325,
 327,
 331,
 333,
 334,
 335,
 336,
 339,
 340,
 341,
 342,
 345,
 350,

In [13]:
len(target_nodwnsmpl_pred_index)

674049

In [14]:
# separating the dataframe into 2 dataframes: 1 for use/train and 1 for prediction comparison

acc_1b_train_downsample = acc_train.iloc[target_nodwnsmpl_index_0and1]
acc_1b_train_downsample = acc_1b_train_downsample.reset_index()
acc_1b_train_downsample = acc_1b_train_downsample.drop('index', axis=1)

target_1b_train_downsample = target_train.iloc[target_nodwnsmpl_index_0and1]
target_1b_train_downsample = target_1b_train_downsample.reset_index()
target_1b_train_downsample = target_1b_train_downsample.drop('index', axis=1)


acc_1b_pred_downsample = acc_train.iloc[target_nodwnsmpl_pred_index]
acc_1b_pred_downsample = acc_1b_pred_downsample.reset_index()
acc_1b_pred_downsample = acc_1b_pred_downsample.drop('index', axis=1)

target_1b_pred_downsample = target_train.iloc[target_nodwnsmpl_pred_index]
target_1b_pred_downsample = target_1b_pred_downsample.reset_index()
target_1b_pred_downsample = target_1b_pred_downsample.drop('index', axis=1)


acc_1b_train_downsample

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.00,0.0000,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68476668,NaN,20000.0,20000.0,20000.0,36 months,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2015,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,550xx,MN,14.67,0.0,Jun-1990,680.0,684.0,0.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,0.0,0.0,21393.800000,21393.80,20000.00,1393.80,0.0,0.00,0.0000,Nov-2016,15681.05,NaN,Mar-2019,654.0,650.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,0.0,2.0,0.0,2.0,18.0,29433.0,63.0,2.0,3.0,13048.0,74.0,94200.0,1.0,0.0,1.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,0.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,0.0,0.0,0.0,2.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,

In [15]:
target_1b_train_downsample

,loan_status_binary
0,0
1,0
2,0
3,0
4,0
...,...
674045,1
674046,1
674047,1
674048,1


In [16]:
# drop unneeded columns

acc_clean = acc_1b_train_downsample.drop(['collection_recovery_fee', 'desc', 'id', 'member_id',
                                          'next_pymnt_d', 'out_prncp', 'out_prncp_inv','pymnt_plan',
                                          'recoveries', 'title', 'total_pymnt_inv',
                                          'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp',
                                          'url', 'hardship_flag', 'hardship_type', 'hardship_reason',
                                          'hardship_status', 'deferral_term', 'hardship_amount',
                                          'hardship_start_date', 'hardship_end_date',
                                          'payment_plan_start_date', 'hardship_length',
                                          'hardship_dpd', 'hardship_loan_status',
                                          'orig_projected_additional_accrued_interest',
                                          'hardship_payoff_balance_amount',
                                          'hardship_last_payment_amount', 'disbursement_method',
                                          'debt_settlement_flag', 'debt_settlement_flag_date',
                                          'settlement_status', 'settlement_date', 'settlement_amount',
                                          'settlement_percentage', 'settlement_term', 'last_pymnt_d',
                                          'last_pymnt_amnt', 'last_fico_range_high',
                                          'last_fico_range_low', 'inq_last_12m', 'inq_last_6mths',
                                          'open_rv_24m', 'open_rv_12m', 'open_il_24m', 'open_il_12m',
                                          'open_acc_6m', 'num_tl_op_past_12m', 'num_tl_90g_dpd_24m',
                                          'num_tl_30dpd', 'num_tl_120dpd_2m', 'num_accts_ever_120_pd',
                                          'sec_app_chargeoff_within_12_mths',
                                          'sec_app_collections_12_mths_ex_med', 'zip_code',
                                          'addr_state'], axis=1)

acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,debt_consolidation,5.91,0.0,Aug-2003,675.0,679.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000.0,20000.0,20000.0,36 months,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2015,debt_consolidation,14.67,0.0,Jun-1990,680.0,684.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000.0,20000.0,20000.0,36 months,8.49,631.26,B,B1,road driver,10+ years,MORTGAGE,85000.0,Not Verified,Dec-2015,major_purchase,17.61,1.0,Feb-1999,705.0,709.0,3.0,NaN,8.0,0.0,826.0,5.7,15.0,w,21538.508977,Mar-2019,0.0,3.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,NaN,8.0,NaN,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000.0,10000.0,10000.0,36 months,6.49,306.45,A,A2,SERVICE MANAGER,6 years,RENT,85000.0,Not Verified,Dec-2015,credit_card,13.07,0.0,Apr-2002,685.0,689.0,NaN,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,NaN,1.0,NaN,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8000.0,8000.0,8000.0,36 months,11.48,263.74,B,B5,Vendor liaison,10+ years,MORTGAGE,42000.0,Not Verified,Dec-2015,credit_card,34.80,0.0,Nov-1994,700.0,704.0,75.0,NaN,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,Nov-2018,0.0,75.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,199696.0,2.0,10.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,5.0,28528.0,9966.0,41.4,0.0,0.0,155.0,253.0,15.0,10.0,1.0,50.0,NaN,10.0,NaN,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,94.

In [17]:
# drop unneeded columns

acc_pred = acc_1b_pred_downsample.drop(['collection_recovery_fee', 'desc', 'id', 'member_id',
                                          'next_pymnt_d', 'out_prncp', 'out_prncp_inv','pymnt_plan',
                                          'recoveries', 'title', 'total_pymnt_inv',
                                          'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp',
                                          'url', 'hardship_flag', 'hardship_type', 'hardship_reason',
                                          'hardship_status', 'deferral_term', 'hardship_amount',
                                          'hardship_start_date', 'hardship_end_date',
                                          'payment_plan_start_date', 'hardship_length',
                                          'hardship_dpd', 'hardship_loan_status',
                                          'orig_projected_additional_accrued_interest',
                                          'hardship_payoff_balance_amount',
                                          'hardship_last_payment_amount', 'disbursement_method',
                                          'debt_settlement_flag', 'debt_settlement_flag_date',
                                          'settlement_status', 'settlement_date', 'settlement_amount',
                                          'settlement_percentage', 'settlement_term', 'last_pymnt_d',
                                          'last_pymnt_amnt', 'last_fico_range_high',
                                          'last_fico_range_low', 'inq_last_12m', 'inq_last_6mths',
                                          'open_rv_24m', 'open_rv_12m', 'open_il_24m', 'open_il_12m',
                                          'open_acc_6m', 'num_tl_op_past_12m', 'num_tl_90g_dpd_24m',
                                          'num_tl_30dpd', 'num_tl_120dpd_2m', 'num_accts_ever_120_pd',
                                          'sec_app_chargeoff_within_12_mths',
                                          'sec_app_collections_12_mths_ex_med', 'zip_code',
                                          'addr_state'], axis=1)

acc_pred

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog
0,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,small_business,16.06,1.0,Dec-1999,715.0,719.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,25679.660000,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,19.0,18005.0,73.0,6472.0,29.0,111800.0,0.0,0.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-2015,home_improvement,10.78,0.0,Aug-2000,695.0,699.0,NaN,NaN,6.0,0.0,7869.0,56.2,18.0,w,22705.924294,Mar-2019,0.0,NaN,1.0,Joint App,71000.0,13.85,Not Verified,0.0,0.0,189699.0,1.0,19.0,10827.0,73.0,2081.0,65.0,14000.0,2.0,5.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,101.0,NaN,10.0,NaN,2.0,3.0,2.0,4.0,6.0,4.0,7.0,3.0,6.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,major_purchase,25.37,1.0,Jun-1998,695.0,699.0,12.0,NaN,12.0,0.0,21929.0,64.5,35.0,w,11740.500000,Mar-2018,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,331730.0,3.0,14.0,73839.0,84.0,9702.0,78.0,34000.0,2.0,1.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,4.0,12.0,1.0,12.0,4.0,6.0,5.0,9.0,10.0,7.0,19.0,6.0,12.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11950.0,11950.0,11950.0,36 months,13.44,405.18,C,C3,Veterinary Tecnician,4 years,RENT,34000.0,Source Verified,Dec-2015,debt_consolidation,10.20,0.0,Oct-1987,690.0,694.0,NaN,NaN,5.0,0.0,8822.0,68.4,6.0,w,13708.948530,May-2017,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,12798.0,1.0,338.0,3976.0,99.0,4522.0,76.0,12900.0,0.0,0.0,0.0,2560.0,844.0,91.0,0.0,0.0,338.0,54.0,32.0,32.0,0.0,36.0,NaN,NaN,NaN,2.0,3.0,2.0,2.0,2.0,4.0,4.0,3.0,5.0,100.0,100.0,0.0,0.0,16900.0,12798.0,9400.0,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1400.0,1400.0,1400.0,36 months,12.88,47.10,C,C2,Logistics Manager,3 years,MORTGAGE,64000.0,Not Verified,Dec-2015,other,34.95,0.0,Jun-1996,700.0,704.0,NaN,NaN,17.0,0.0,37828.0,67.2,24.0,w,1575.160698,Sep-2018,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,325695.0,3.0,2.0,37430.0,67.0,7386.0,67.0,56300.0,0.0,2.0,4.0,19159.0,7940.0,77.0,0.0,0.0,46.0,234.0,18.0,2.0,4.0,28.0,NaN,18.0,NaN,7.0,11.0,8.0,11.0,3.0,12.0,17.0,11

In [18]:
datasets_2 = [acc_clean, acc_pred]
              
targets_2 = [target_1b_train_downsample, target_1b_pred_downsample]

In [19]:
for dataset in datasets_2:
    
    # if employment length is 10+ years, insert 10 years
    dataset['emp_length'] = np.where(dataset['emp_length']=='10+ years',
                                       '10 years', dataset['emp_length'])

    # if employment length is 1 year , insert 1 years
    dataset['emp_length'] = np.where(dataset['emp_length']=='1 year',
                                       '1 years', dataset['emp_length'])

    # if employment length is <1, insert 0
    dataset['emp_length'] = np.where(dataset['emp_length']=='< 1 year',
                                       '0 years', dataset['emp_length'])

    # if employment length is null, insert -1
    dataset['emp_length'] = np.where(dataset['emp_length'].isnull(),
                                       '-1 years', dataset['emp_length'])
    # removing ' years' from the column
    dataset['emp_length'] = dataset['emp_length'].str.replace(' years', '', regex=False)

    # Convert to numeric type
    dataset['emp_length'] = pd.to_numeric(dataset['emp_length'])
    
    # Check if columns with dtype object should be ordinal
    # Get columns with string values
    cols_w_strings = dataset.loc[:, dataset.dtypes == object].columns
    
    # Home Ownership
    # Assigning values to home_ownership status
    dataset.loc[dataset['home_ownership']=='OWN', 'home_ownership'] = 3
    dataset.loc[dataset['home_ownership']=='MORTGAGE', 'home_ownership'] = 2
    dataset.loc[dataset['home_ownership']=='ANY', 'home_ownership'] = 2
    dataset.loc[dataset['home_ownership']=='OTHER', 'home_ownership'] = 1
    dataset.loc[dataset['home_ownership']=='RENT', 'home_ownership'] = 0
    dataset.loc[dataset['home_ownership']=='NONE', 'home_ownership'] = 0
    dataset['home_ownership'] = dataset['home_ownership'].astype(int)
    
    # Grade
    # Grouping E, F, and G into F
    dataset['grouped_grade_NO_EG'] = dataset['grade']
    dataset['grouped_grade_NO_EG'] = dataset['grouped_grade_NO_EG'].str.replace('E', 'F',
                                                                                    regex=False)
    dataset['grouped_grade_NO_EG'] = dataset['grouped_grade_NO_EG'].str.replace('G', 'F',
                                                                                    regex=False)
    
    # replacing letter grades with numbers
    # get sorted list of grades
    grade_list = sorted(list(dataset['grade'].unique()))
    i = 1
    for grades in grade_list:
        dataset.loc[dataset['grade']==grades, 'grade'] = i
        i+=1
    dataset['grade'] = dataset['grade'].astype(int)
    
    # replacing letter grades with numbers for grouped grades
    # get sorted list of grades
    grade_list = sorted(list(dataset['grouped_grade_NO_EG'].unique()))
    i = 1
    for grades in grade_list:
        dataset.loc[dataset['grouped_grade_NO_EG']==grades, 'grouped_grade_NO_EG'] = i
        i+=1
    dataset['grouped_grade_NO_EG'] = dataset['grouped_grade_NO_EG'].astype(int)
    
    # Subgrade
    # grouping subgrades 1, 2 vs 3, 4, 5
    dataset['grouped_subgrade_12_345'] = dataset['sub_grade']
    dataset['grouped_subgrade_12_345'] = dataset\
    ['grouped_subgrade_12_345'].str.replace('2','1',regex=False)
    dataset['grouped_subgrade_12_345'] = dataset\
    ['grouped_subgrade_12_345'].str.replace('3','2', regex=False)
    dataset['grouped_subgrade_12_345'] = dataset\
    ['grouped_subgrade_12_345'].str.replace('4','2', regex=False)
    dataset['grouped_subgrade_12_345'] = dataset\
    ['grouped_subgrade_12_345'].str.replace('5','2', regex=False)

    # grouping subgrades 1, 2, 3 vs 4, 5
    dataset['grouped_subgrade_123_45'] = dataset['sub_grade']
    dataset['grouped_subgrade_123_45'] = dataset\
    ['grouped_subgrade_123_45'].str.replace('2','1', regex=False)
    dataset['grouped_subgrade_123_45'] = dataset\
    ['grouped_subgrade_123_45'].str.replace('3','1', regex=False)
    dataset['grouped_subgrade_123_45'] = dataset\
    ['grouped_subgrade_123_45'].str.replace('4','2', regex=False)
    dataset['grouped_subgrade_123_45'] = dataset\
    ['grouped_subgrade_123_45'].str.replace('5','2', regex=False)
    dataset['grouped_subgrade_123_45'].value_counts()

    # grouping Subgrades
    dataset['grouped_subgrade'] = dataset['sub_grade']
    dataset['grouped_subgrade'] = dataset['grouped_subgrade'].str.replace('E','F', regex=False)
    dataset['grouped_subgrade'] = dataset['grouped_subgrade'].str.replace('G','F', regex=False)
    
    # replacing subgrades with numbers
    # get sorted list of subgrades
    subgrade_list = sorted(list(dataset['sub_grade'].unique()))
    i = 1
    for subgrades in subgrade_list:
        dataset.loc[dataset['sub_grade']==subgrades, 'sub_grade'] = i
        i+=1
    dataset['sub_grade'] = dataset['sub_grade'].astype(int)
    
    # replacing subgrades with numbers
    # get sorted list of subgrades
    subgrade_list = sorted(list(dataset['grouped_subgrade_12_345'].unique()))
    i = 1
    for subgrades in subgrade_list:
        dataset.loc[dataset['grouped_subgrade_12_345']==subgrades, 'grouped_subgrade_12_345'] = i
        i+=1
    dataset['grouped_subgrade_12_345'] = dataset['grouped_subgrade_12_345'].astype(int)
    
    # replacing subgrades with numbers
    # get sorted list of subgrades
    subgrade_list = sorted(list(dataset['grouped_subgrade_123_45'].unique()))
    i = 1
    for subgrades in subgrade_list:
        dataset.loc[dataset['grouped_subgrade_123_45']==subgrades, 'grouped_subgrade_123_45'] = i
        i+=1
    dataset['grouped_subgrade_123_45'] = dataset['grouped_subgrade_123_45'].astype(int)
    
    # replacing subgrades with numbers
    # get sorted list of subgrades
    subgrade_list = sorted(list(dataset['grouped_subgrade'].unique()))
    i = 1
    for subgrades in subgrade_list:
        dataset.loc[dataset['grouped_subgrade']==subgrades, 'grouped_subgrade'] = i
        i+=1
    dataset['grouped_subgrade'] = dataset['grouped_subgrade'].astype(int)
    
    # Earliest Credit Line
    # converting the column to datetime format
    dataset['earliest_cr_line_dt'] = pd.DatetimeIndex(dataset['earliest_cr_line'])
    
    # creating a new column that will show the number of months that credit has existed for customer
    dataset['duration_of_credit_months'] = (pd.DatetimeIndex(dataset['issue_d']).year -\
                                            pd.DatetimeIndex(dataset['earliest_cr_line']).year)*12 +\
    (pd.DatetimeIndex(dataset['issue_d']).month -\
     pd.DatetimeIndex(dataset['earliest_cr_line']).month)

    # dropping the columns that helped create the new column
    dataset = dataset.drop(['earliest_cr_line_dt', 'earliest_cr_line'], axis=1)
    
    # Last Credit Pulled Date
    # dropping the column bc it is used post-funding
    dataset = dataset.drop('last_credit_pull_d', axis=1)
    
    # Fico Score
    dataset['fico_score'] = (dataset['fico_range_high'] + dataset['fico_range_low'])/2
    dataset['sec_app_fico_score'] = (dataset['sec_app_fico_range_high'] +\
                                       dataset['sec_app_fico_range_low'])/2
    dataset = dataset.drop(['fico_range_high', 'fico_range_low',
                                        'sec_app_fico_range_high',
                                        'sec_app_fico_range_low'], axis=1)
    # Verification status
    # if the income is verified, change to 1. If not verified, change to 0
    dataset.loc[((dataset['verification_status']=='Source Verified') |\
                 (dataset['verification_status']=='Verified')), 'verification_status'] = 1
    dataset.loc[dataset['verification_status']=='Not Verified', 'verification_status'] = 0
    
    # change dtype to int
    dataset[['verification_status']].astype(int)
    
    dataset.loc[dataset['application_type']=='Joint App', 'application_type'] = 2
    dataset.loc[dataset['application_type']=='Individual', 'application_type'] = 1
    dataset['application_type'] = dataset['application_type'].astype(int)
    
    '''
    for verification_status_joint where there are values in cols that start with
    sec_app, but verification_status_joint is null
    '''
    # add in 'not verified' where applicable
    dataset.loc[((dataset['verification_status_joint'].notnull()) &\
                 (dataset['application_type']==1)), 'verification_status_joint'] = 'Not Verified'
    dataset.loc[((dataset['verification_status_joint'].isnull()) &\
                 (dataset['application_type']==2)), 'verification_status_joint'] = 'Not Verified'
    dataset.loc[((dataset['verification_status_joint'].isnull()) &\
                 (dataset['application_type']==1)), 'verification_status_joint'] = 'Individual'
    
    # if the income is verified, change to 1. If not verified, change to 0
    dataset.loc[((dataset['verification_status_joint']=='Source Verified') |\
                      (dataset['verification_status_joint']=='Verified')),
                      'verification_status_joint'] = 1
    dataset.loc[dataset['verification_status_joint']=='Individual',
                      'verification_status_joint'] = -1
    dataset.loc[dataset['verification_status_joint']=='Not Verified',
                      'verification_status_joint'] = 0
    dataset['verification_status_joint'] = dataset['verification_status_joint'].astype(int)
    
    # Reducing Income if not Verified
    dataset['annual_inc_reduced'] = dataset['annual_inc']
    dataset.loc[dataset['verification_status']==0,
                'annual_inc_reduced'] = dataset['annual_inc'] * 0.7
    dataset['annual_inc_joint_reduced'] = dataset['annual_inc_joint']
    dataset.loc[dataset['verification_status_joint']==0,
                'annual_inc_joint_reduced'] = dataset['annual_inc_joint'] * 0.7
    
    # Joint Earliest Credit Line
    # converting column to datetime
    dataset['sec_app_earliest_cr_line_dt'] = pd.DatetimeIndex(dataset['sec_app_earliest_cr_line'])

    # creating a new column that will show the number of months
    #that credit has existed for joint customer
    dataset['sec_app_duration_of_credit_months']=(pd.DatetimeIndex\
                                                  (dataset['issue_d']).year - pd.DatetimeIndex\
                                                  (dataset['sec_app_earliest_cr_line']).year)*12 +\
    (pd.DatetimeIndex(dataset['issue_d']).month -\
     pd.DatetimeIndex(dataset['sec_app_earliest_cr_line']).month)

    # converting null values to -1
    dataset.loc[dataset['sec_app_duration_of_credit_months'].isnull(),
                'sec_app_duration_of_credit_months'] = -1
    
    dataset = dataset.drop(['sec_app_earliest_cr_line_dt', 'sec_app_earliest_cr_line',
                            'issue_d', 'sec_app_earliest_cr_line',
                            'sec_app_earliest_cr_line_dt'], axis=1)
    
    cols_w_mths_since = [col for col in dataset.columns if col.startswith('mths_since_')]
    for col in cols_w_mths_since:
        dataset.loc[dataset[col].isnull(), col] = 10000
    
    # Get cols with null values
    cols_w_null = []

    # function that gives list of columns that contain null values
    null_cols(dataset, cols_w_null)

    # Fill in null values
    # creates a list of the index where the column == 0
    list(dataset[dataset['emp_length']==0].index)
    
    # getting the columns where there are 0s
    cols_w_0s_and_null = []
    for col in cols_w_null:
        # if the list is not empty, then there are values == 0
        if list(dataset[dataset[col]==0].index)!=[]:
            cols_w_0s_and_null.append(col)
        for col_0 in cols_w_0s_and_null:
            dataset.loc[dataset[col_0].isnull(), col_0] = -1
        # rerun to get new cols w null values

    cols_w_null = []
    # function that gives list of columns that contain null values
    null_cols(dataset, cols_w_null)

    # Employment
    dataset.loc[dataset['emp_title'].isnull(), 'emp_title'] = 'Unknown'
    
    # Employment length
    emplength_mean = dataset['emp_length'].mean()
    dataset.loc[(dataset['emp_length'].isnull()) &\
                (dataset['emp_title'].notnull()), 'emp_length'] = emplength_mean
    dataset.loc[(dataset['emp_length'].isnull()) &\
                (dataset['emp_title'].isnull()), 'emp_length'] = -1
    dataset = dataset.drop(['emp_title'], axis=1)
    
    # joint
    # if no secondary app, convert to -1
    dataset.loc[dataset['annual_inc_joint'].isnull(), 'annual_inc_joint'] = -1
    dataset.loc[dataset['annual_inc_joint_reduced'].isnull(),'annual_inc_joint_reduced'] = -1
    dataset.loc[dataset['sec_app_fico_score'].isnull(), 'sec_app_fico_score'] = -1
    
    # others
    dataset.loc[dataset['annual_inc'].isnull(), 'annual_inc'] = -1
    dataset.loc[dataset['mo_sin_old_rev_tl_op'].isnull(), 'mo_sin_old_rev_tl_op'] = 10000
    dataset.loc[dataset['duration_of_credit_months'].isnull(),
                      'duration_of_credit_months'] = -1
    dataset.loc[dataset['sec_app_duration_of_credit_months'].isnull(),
                      'sec_app_duration_of_credit_months'] = -1
    
    dataset.loc[dataset['term'].str.contains('60', na=False), 'term'] = 1
    dataset.loc[dataset['term'].str.contains('36', na=False), 'term'] = 0
    dataset['term'].astype(int).dtypes
    
    # total accounts
    cols_w_acc = [col for col in dataset.columns if 'acc' in col]
    
    # using the open_acc as a filler for the null values for total_acc
    dataset.loc[dataset['total_acc'].isnull(), 'total_acc'] = dataset['open_acc']
    
    cols_w_null = []
    # function that gives list of columns that contain null values
    null_cols(dataset, cols_w_null)
    cols_w_null_list = list(dataset[cols_w_null].columns)
    
    # getting the columns where there are 0s
    for col in cols_w_null_list:
        dataset.loc[dataset[col].isnull(), col] = -1
        
    cols_w_null = []
    # function that gives list of columns that contain null values
    null_cols(dataset, cols_w_null)
    
    num_cols = [col for col in dataset.columns if '2' in col]
    num_cols.remove('grouped_subgrade_12_345')
    num_cols.remove('grouped_subgrade_123_45')
    dataset = dataset.drop(num_cols, axis=1)
    
    # Income : * Ratios
    dataset[(dataset['application_type']==1) &\
            (dataset['annual_inc_joint']==dataset['annual_inc'])][['annual_inc_joint', 'annual_inc']]
    
    # Giving weight to loan amount
    # creating a new column
    dataset['weighted_loan_amnt'] = dataset['loan_amnt']

    # multiplying the new column by its perceived rank according to FICO score ranges
    dataset.loc[dataset['fico_score']<=560,
                  'weighted_loan_amnt'] = dataset['weighted_loan_amnt'] * 5
    dataset.loc[((dataset['fico_score']>560) &\
                   (dataset['fico_score']<=650)),
                  'weighted_loan_amnt'] = dataset['weighted_loan_amnt'] * 4
    dataset.loc[((dataset['fico_score']>650) &\
                   (dataset['fico_score']<=700)),
                  'weighted_loan_amnt'] = dataset['weighted_loan_amnt'] * 3
    dataset.loc[((dataset['fico_score']>700) &\
                   (dataset['fico_score']<=750)),
                  'weighted_loan_amnt'] = dataset['weighted_loan_amnt'] * 2
    dataset.loc[dataset['fico_score']>750,
                  'weighted_loan_amnt'] = dataset['weighted_loan_amnt'] * 1
    dataset['weighted_loan_amnt']
    
    # Giving weight to funded amount
    # creating a new column
    dataset['weighted_funded_amnt'] = dataset['funded_amnt']

    # multiplying the new column by its perceived rank according to FICO score ranges
    dataset.loc[dataset['fico_score']<=560,
                  'weighted_funded_amnt'] = dataset['weighted_funded_amnt'] * 5
    dataset.loc[((dataset['fico_score']>560) &\
                   (dataset['fico_score']<=650)),
                  'weighted_funded_amnt'] = dataset['weighted_funded_amnt'] * 4
    dataset.loc[((dataset['fico_score']>650) &\
                   (dataset['fico_score']<=700)),
                  'weighted_funded_amnt'] = dataset['weighted_funded_amnt'] * 3
    dataset.loc[((dataset['fico_score']>700) &\
                   (dataset['fico_score']<=750)),
                  'weighted_funded_amnt'] = dataset['weighted_funded_amnt'] * 2
    dataset.loc[dataset['fico_score']>750,
                  'weighted_funded_amnt'] = dataset['weighted_funded_amnt'] * 1
    dataset['weighted_funded_amnt']
    
    # Giving weight to installment
    # creating a new column
    dataset['weighted_installment'] = dataset['installment']

    # multiplying the new column by its perceived rank according to FICO score ranges
    dataset.loc[dataset['fico_score']<=560,
                  'weighted_installment'] = dataset['weighted_installment'] * 5
    dataset.loc[((dataset['fico_score']>560) &\
                   (dataset['fico_score']<=650)),
                  'weighted_installment'] = dataset['weighted_installment'] * 4
    dataset.loc[((dataset['fico_score']>650) &\
                   (dataset['fico_score']<=700)),
                  'weighted_installment'] = dataset['weighted_installment'] * 3
    dataset.loc[((dataset['fico_score']>700) &\
                   (dataset['fico_score']<=750)),
                  'weighted_installment'] = dataset['weighted_installment'] * 2
    dataset.loc[dataset['fico_score']>750,
                  'weighted_installment'] = dataset['weighted_installment'] * 1
    dataset['weighted_installment']
    
    # 1st borrower income
    # Income to loan amount ratio
    dataset['ratio_inc_loan'] = dataset['annual_inc'] / dataset['loan_amnt']
    dataset['ratio_inc_weightedloan'] = dataset['annual_inc'] / dataset['weighted_loan_amnt']

    # income reduced to loan amount ratio
    dataset['ratio_redinc_loan'] = dataset['annual_inc_reduced'] / dataset['loan_amnt']
    dataset['ratio_redinc_weightedloan'] = dataset['annual_inc_reduced'] /\
    dataset['weighted_loan_amnt']

    # Income to funded amount ratio
    dataset['ratio_inc_funded'] = dataset['annual_inc'] / dataset['funded_amnt']
    dataset['ratio_inc_weightedfunded'] = dataset['annual_inc'] / dataset['weighted_funded_amnt']

    # income reduced to funded amount ratio
    dataset['ratio_redinc_funded'] = dataset['annual_inc_reduced'] /\
    dataset['funded_amnt']
    dataset['ratio_redinc_weightedfunded'] = dataset['annual_inc_reduced'] /\
    dataset['weighted_funded_amnt']

    # installment to income ratio
    dataset['ratio_install_inc'] = dataset['annual_inc'] / dataset['installment']
    dataset['ratio_weightedinstall_inc'] = dataset['annual_inc'] / dataset['weighted_installment']

    # installment to reduced income ratio
    dataset['ratio_install_redinc'] = dataset['annual_inc_reduced'] /\
    dataset['installment']
    dataset['ratio_weightedinstall_redinc'] = dataset['annual_inc_reduced'] /\
    dataset['weighted_installment']
    
    # joint borrower income
    # Income to loan amount ratio
    dataset['ratio_incjoint_loan'] = dataset['annual_inc_joint'] / dataset['loan_amnt']
    dataset['ratio_incjoint_weightedloan'] = dataset['annual_inc_joint'] /\
    dataset['weighted_loan_amnt']

    # income reduced to loan amount ratio
    dataset['ratio_redincjoint_loan'] = dataset['annual_inc_joint_reduced'] /\
    dataset['loan_amnt']
    dataset['ratio_redincjoint_weightedloan'] = dataset['annual_inc_joint_reduced'] /\
    dataset['weighted_loan_amnt']

    # Income to funded amount ratio
    dataset['ratio_incjoint_funded'] = dataset['annual_inc_joint'] /\
    dataset['funded_amnt']
    dataset['ratio_incjoint_weightedfunded'] = dataset['annual_inc_joint'] /\
    dataset['weighted_funded_amnt']

    # income reduced to funded amount ratio
    dataset['ratio_redincjoint_funded'] = dataset['annual_inc_joint_reduced'] /\
    dataset['funded_amnt']
    dataset['ratio_redincjoint_weightedfunded'] = dataset['annual_inc_joint_reduced'] /\
    dataset['weighted_funded_amnt']

    # Income to installment ratio
    dataset['ratio_incjoint_installment'] = dataset['annual_inc_joint']/\
    dataset['installment']
    dataset['ratio_incjoint_weightedinstallment'] = dataset['annual_inc_joint']/\
    dataset['weighted_installment']

    # Reduced income to installment ratio
    dataset['ratio_redincjoint_installment'] = dataset['annual_inc_joint_reduced']/\
    dataset['installment']
    dataset['ratio_redincjoint_weightedinstallment'] = dataset['annual_inc_joint_reduced']/\
    dataset['weighted_installment']
    
    # Loan Amount vs Funded Amount
    dataset['loan_minus_funded'] = dataset['loan_amnt'] - dataset['funded_amnt']
    dataset['ratio_loan_funded'] = dataset['loan_amnt'] / dataset['funded_amnt']

In [20]:
acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,earliest_cr_line_dt,duration_of_credit_months
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,3,14,leadman,10,2,55000.0,Not Verified,Dec-2015,debt_consolidation,5.91,0.0,Aug-2003,675.0,679.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6,6,14,2003-08-01,148.0
1,20000.0,20000.0,20000.0,36 months,9.17,637.58,2,7,Vice President of Recruiting Operations,10,2,180000.0,Not Verified,Dec-2015,debt_consolidation,14.67,0.0,Jun-1990,680.0,684.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,3,7,1990-06-01,306.0
2,20000.0,20000.0,20000.0,36 months,8.49,631.26,2,6,road driver,10,2,85000.0,Not Verified,Dec-2015,major_purchase,17.61,1.0,Feb-1999,705.0,709.0,3.0,NaN,8.0,0.0,826.0,5.7,15.0,w,21538.508977,Mar-2019,0.0,3.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,NaN,8.0,NaN,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,3,6,1999-02-01,202.0
3,10000.0,10000.0,10000.0,36 months,6.49,306.45,1,2,SERVICE MANAGER,6,0,85000.0,Not Verified,Dec-2015,credit_card,13.07,0.0,Apr-2002,685.0,689.0,NaN,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,NaN,1.0,NaN,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,2,2002-04-01,164.0
4,8000.0,8000.0,8000.0,36 months,11.48,263.74,2,10,Vendor liaison,10,2,42000.0,Not Verified,Dec-2015,credit_card,34.80,0.0,Nov-1994,700.0,704.0,75.0,NaN,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,Nov-2018,0.0,75.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,199696.0,2.0

In [21]:
acc_pred

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,earliest_cr_line_dt,duration_of_credit_months
0,24700.0,24700.0,24700.0,36 months,11.99,820.28,3,11,Engineer,10,2,65000.0,Not Verified,Dec-2015,small_business,16.06,1.0,Dec-1999,715.0,719.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,25679.660000,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,19.0,18005.0,73.0,6472.0,29.0,111800.0,0.0,0.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,5,5,11,1999-12-01,192.0
1,20000.0,20000.0,20000.0,60 months,10.78,432.66,2,9,truck driver,10,2,63000.0,Not Verified,Dec-2015,home_improvement,10.78,0.0,Aug-2000,695.0,699.0,NaN,NaN,6.0,0.0,7869.0,56.2,18.0,w,22705.924294,Mar-2019,0.0,NaN,1.0,Joint App,71000.0,13.85,Not Verified,0.0,0.0,189699.0,1.0,19.0,10827.0,73.0,2081.0,65.0,14000.0,2.0,5.0,6.0,31617.0,2737.0,55.9,0.0,0.0,125.0,184.0,14.0,14.0,5.0,101.0,NaN,10.0,NaN,2.0,3.0,2.0,4.0,6.0,4.0,7.0,3.0,6.0,100.0,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4,4,9,2000-08-01,184.0
2,10400.0,10400.0,10400.0,60 months,22.45,289.91,6,26,Contract Specialist,3,2,104433.0,Source Verified,Dec-2015,major_purchase,25.37,1.0,Jun-1998,695.0,699.0,12.0,NaN,12.0,0.0,21929.0,64.5,35.0,w,11740.500000,Mar-2018,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,331730.0,3.0,14.0,73839.0,84.0,9702.0,78.0,34000.0,2.0,1.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,4.0,12.0,1.0,12.0,4.0,6.0,5.0,9.0,10.0,7.0,19.0,6.0,12.0,96.6,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,11,11,21,1998-06-01,210.0
3,11950.0,11950.0,11950.0,36 months,13.44,405.18,3,13,Veterinary Tecnician,4,0,34000.0,Source Verified,Dec-2015,debt_consolidation,10.20,0.0,Oct-1987,690.0,694.0,NaN,NaN,5.0,0.0,8822.0,68.4,6.0,w,13708.948530,May-2017,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,12798.0,1.0,338.0,3976.0,99.0,4522.0,76.0,12900.0,0.0,0.0,0.0,2560.0,844.0,91.0,0.0,0.0,338.0,54.0,32.0,32.0,0.0,36.0,NaN,NaN,NaN,2.0,3.0,2.0,2.0,2.0,4.0,4.0,3.0,5.0,100.0,100.0,0.0,0.0,16900.0,12798.0,9400.0,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6,5,13,1987-10-01,338.0
4,1400.0,1400.0,1400.0,36 months,12.88,47.10,3,12,Logistics Manager,3,2,64000.0,Not Verified,Dec-2015,other,34.95,0.0,Jun-1996,700.0,704.0,NaN,NaN,17.0,0.0,37828.0,67.2,24.0,w,1575.160698,Sep-2018,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.

# Dummification

In [22]:
# getting a list of cols w dtype object

cols_w_obj = list(acc_clean.select_dtypes(include=['object']).columns)

cols_w_obj

['term',
 'emp_title',
 'verification_status',
 'issue_d',
 'purpose',
 'earliest_cr_line',
 'initial_list_status',
 'last_credit_pull_d',
 'application_type',
 'verification_status_joint',
 'sec_app_earliest_cr_line']

In [1]:
acc_clean[['emp_title']]

NameError: name 'acc_clean' is not defined

In [101]:
# dummification

acc_dum = pd.get_dummies(acc_clean, drop_first=True)

acc_dum

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w
0,3600.0,3600.0,3600.0,0,13.99,123.03,3,14,10.0,2,55000.0,0,5.91,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,4421.723917,30.0,1.0,1,-1.0,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,20701.0,1506.0,37.2,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3,6,6,14,148.0,677.0,-1.0,38500.0,-1.0,-1.0,10800.0,10800.0,369.09,15.277778,5.092593,10.694444,3.564815,15.277778,5.092593,10.694444,3.564815,447.045436,149.015145,312.931805,104.310602,-0.000278,-0.000093,-0.000278,-0.000093,-0.000278,-0.000093,-0.000278,-0.000093,-0.008128,-0.002709,-0.008128,-0.002709,0.0,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,20000.0,20000.0,20000.0,0,9.17,637.58,2,7,10.0,2,180000.0,0,14.67,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,21393.800000,10000.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,30030.0,0.0,102.9,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,3,3,7,306.0,682.0,-1.0,126000.0,-1.0,-1.0,60000.0,60000.0,1912.74,9.000000,3.000000,6.300000,2.100000,9.000000,3.000000,6.300000,2.100000,282.317513,94.105838,197.622259,65.874086,-0.000050,-0.000017,-0.000050,-0.000017,-0.000050,-0.000017,-0.000050,-0.000017,-0.001568,-0.000523,-0.001568,-0.000523,0.0,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,20000.0,20000.0,20000.0,0,8.49,631.26,2,6,10.0,2,85000.0,0,17.61,3.0,10000.0,8.0,0.0,826.0,5.7,15.0,21538.508977,3.0,1.0,1,-1.0,-1.0,

# Scaling the dataset

In [102]:
# we are preserving total_pymnt to use as our 

acc_totalpayment = copy.deepcopy(acc_dum[['total_pymnt']])

acc_dum = acc_dum.drop('total_pymnt', axis=1)

In [103]:
# creating copies

import copy

acc_full = copy.deepcopy(acc_dum)

In [104]:
# full scaling

from sklearn.preprocessing import MinMaxScaler

acc_dum_cols = list(acc_dum.columns)

for col in acc_dum_cols:
    acc_full[col] = MinMaxScaler().fit_transform(np.array(acc_full[col]).reshape(-1,1))
    
acc_full

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w
0,0.078481,0.078481,0.09,0.0,0.338006,0.068955,0.333333,0.382353,1.000000,0.666667,0.005000,0.0,0.00691,0.0030,1.0000,0.087912,0.016129,0.001076,0.158247,0.080460,0.0030,0.0,0.0,0.0,0.0,0.0,0.066667,0.000852,0.018113,0.062500,0.0021,0.004690,0.066190,0.002400,0.202312,0.00093,0.137931,0.0250,0.025877,0.002691,0.149102,0.000004,0.205517,0.012603,0.009112,0.014652,0.038462,0.0004,0.0069,0.0004,0.0069,0.083333,0.078125,0.046875,0.084507,0.02500,0.059524,0.077519,0.108696,0.087912,0.771287,0.009901,0.083333,0.016129,0.017805,0.002881,0.003159,0.006535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50,0.384615,0.384615,0.541667,0.174678,0.276008,0.0,0.003500,0.0,0.0,0.083893,0.083893,0.069045,0.002619,0.002619,0.001833,0.001833,0.002619,0.002619,0.001833,0.001833,0.002575,0.002575,0.001803,0.001803,0.000008,0.000005,0.000011,0.000007,0.000008,0.000005,0.000011,0.000007,0.000027,0.000009,0.000039,0.000013,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.493671,0.493671,0.50,0.0,0.150312,0.369388,0.166667,0.176471,1.000000,0.666667,0.016365,0.0,0.01567,0.0049,1.0000,0.142857,0.016129,0.033993,0.440722,0.160920,1.0000,0.0,0.0,0.0,0.0,0.0,0.066667,0.000001,0.045044,0.062500,0.0018,0.027708,0.114490,0.043322,0.433526,0.00942,0.068966,0.0125,0.037538,0.000002,0.405543,0.000004,0.197241,0.030406,0.025057,0.040293,0.096154,0.0012,1.0000,0.0010,1.0000,0.138889,0.109375,0.078125,0.084507,0.05000,0.119048,0.131783,0.152174,0.142857,0.963366,1.000000,0.083333,0.016129,0.038885,0.043424,0.041449,0.022100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.153846,0.153846,0.250000,0.359906,0.297240,0.0,0.011455,0.0,0.0,0.496644,0.496644,0.369

In [105]:
col_names = list(acc_full.columns)
col_names.append('total_pymnt')

acc_full = pd.DataFrame(np.concatenate((acc_full, acc_totalpayment), axis=1),
                             columns=col_names)
acc_full

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w,total_pymnt
0,0.078481,0.078481,0.09,0.0,0.338006,0.068955,0.333333,0.382353,1.000000,0.666667,0.005000,0.0,0.00691,0.0030,1.0000,0.087912,0.016129,0.001076,0.158247,0.080460,0.0030,0.0,0.0,0.0,0.0,0.0,0.066667,0.000852,0.018113,0.062500,0.0021,0.004690,0.066190,0.002400,0.202312,0.00093,0.137931,0.0250,0.025877,0.002691,0.149102,0.000004,0.205517,0.012603,0.009112,0.014652,0.038462,0.0004,0.0069,0.0004,0.0069,0.083333,0.078125,0.046875,0.084507,0.02500,0.059524,0.077519,0.108696,0.087912,0.771287,0.009901,0.083333,0.016129,0.017805,0.002881,0.003159,0.006535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50,0.384615,0.384615,0.541667,0.174678,0.276008,0.0,0.003500,0.0,0.0,0.083893,0.083893,0.069045,0.002619,0.002619,0.001833,0.001833,0.002619,0.002619,0.001833,0.001833,0.002575,0.002575,0.001803,0.001803,0.000008,0.000005,0.000011,0.000007,0.000008,0.000005,0.000011,0.000007,0.000027,0.000009,0.000039,0.000013,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4421.723917
1,0.493671,0.493671,0.50,0.0,0.150312,0.369388,0.166667,0.176471,1.000000,0.666667,0.016365,0.0,0.01567,0.0049,1.0000,0.142857,0.016129,0.033993,0.440722,0.160920,1.0000,0.0,0.0,0.0,0.0,0.0,0.066667,0.000001,0.045044,0.062500,0.0018,0.027708,0.114490,0.043322,0.433526,0.00942,0.068966,0.0125,0.037538,0.000002,0.405543,0.000004,0.197241,0.030406,0.025057,0.040293,0.096154,0.0012,1.0000,0.0010,1.0000,0.138889,0.109375,0.078125,0.084507,0.05000,0.119048,0.131783,0.152174,0.142857,0.963366,1.000000,0.083333,0.016129,0.038885,0.043424,0.041449,0.022100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.153846,0.153846,0.250000,0.359906,0.297240,0.0,0.011455,0.0,0.0

## Downsampling the dataset

In [106]:
target_1b_train_downsample[['loan_status_binary']]

,loan_status_binary
0,0
1,0
2,0
3,0
4,0
...,...
674045,1
674046,1
674047,1
674048,1


In [107]:
from sklearn.utils import resample

label = target_1b_train_downsample['loan_status_binary']

target_mean = label.mean()

target_fullypaid = label == 0

subsample_ind = resample(label[target_fullypaid].index.values, \
                         replace=False,
                         n_samples=int(len(target_1b_train_downsample.index)*target_mean))

combined = np.concatenate([subsample_ind, label[~target_fullypaid].index.values])

acc_subsample = acc_full.iloc[combined]
target_subsample = target_1b_train_downsample[['loan_status_binary']].iloc[combined]

acc_subsample = acc_subsample.reset_index()
acc_subsample = acc_subsample.drop('index', axis=1)

target_subsample = target_subsample.reset_index()
target_subsample = target_subsample.drop('index', axis=1)

target_subsample.mean()

loan_status_binary    0.5
dtype: float64

In [108]:
acc_subsample

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w,total_pymnt
0,0.367089,0.367089,0.374375,0.0,0.485202,0.312740,0.500000,0.441176,1.000000,1.000000,0.009546,0.0,0.00838,0.0022,1.0000,0.087912,0.016129,0.006397,0.255670,0.103448,0.0022,0.0,0.0,0.000000,0.000000,0.0,0.066667,0.000001,0.002569,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,0.00338,0.000000,0.0000,0.003671,0.001902,0.350507,0.000004,0.255172,0.010602,0.020501,0.032967,0.038462,0.0008,1.0000,0.0004,0.0022,0.083333,0.078125,0.046875,0.070423,0.03125,0.071429,0.085271,0.108696,0.087912,1.000000,1.000000,0.083333,0.016129,0.004188,0.007644,0.012501,0.003807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.75,0.461538,0.461538,0.625000,0.213365,0.382166,0.000000,0.006682,0.000000,0.000000,0.244966,0.244966,0.207802,0.001200,0.001800,0.000840,0.001260,0.001200,0.001800,0.000840,0.001260,0.001119,0.001678,0.000783,0.001175,0.000009,0.000006,0.000013,0.000008,0.000009,0.000006,0.000013,0.000008,0.000028,0.000009,0.000040,0.000013,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16643.221785
1,0.240506,0.240506,0.250000,0.0,0.182243,0.185497,0.166667,0.205882,1.000000,0.666667,0.007273,1.0,0.00658,0.0052,0.0045,0.054945,0.032258,0.001814,0.197423,0.097701,0.0075,0.0,0.0,0.000000,0.000000,0.0,0.066667,0.004856,0.018869,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,0.00125,0.000000,0.0000,0.047174,0.014004,0.149493,0.000004,0.175172,0.015303,0.059226,0.095238,0.115385,0.0025,0.0075,0.0022,0.0075,0.083333,0.046875,0.046875,0.084507,0.02500,0.035714,0.069767,0.065217,0.054945,0.868317,0.504950,0.166667,0.016129,0.018550,0.004217,0.016449,0.005375,0.000000,0.000000,0.000

In [109]:
target_subsample

,loan_status_binary
0,0
1,0
2,0
3,0
4,0
...,...
269355,1
269356,1
269357,1
269358,1


In [110]:
target_subsample.value_counts()

loan_status_binary
0                     134680
1                     134680
dtype: int64

In [111]:
acc_subsample.to_csv("acc_2_train_feateng_dwnsmpl.csv.gz",
                index=False,
                compression="gzip")

target_subsample.to_csv("target_2_train_feateng_dwnsmpl.csv.gz",
                        index=False,
                        compression="gzip")

In [112]:
grade_cols = [col for col in acc_subsample.columns if 'grade' in col]

grade_cols

['grade',
 'sub_grade',
 'grouped_grade_NO_EG',
 'grouped_subgrade_12_345',
 'grouped_subgrade_123_45',
 'grouped_subgrade']

In [113]:
acc_subsample_nogrades = acc_subsample.drop(grade_cols, axis=1)

acc_subsample_nogrades

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w,total_pymnt
0,0.367089,0.367089,0.374375,0.0,0.485202,0.312740,1.000000,1.000000,0.009546,0.0,0.00838,0.0022,1.0000,0.087912,0.016129,0.006397,0.255670,0.103448,0.0022,0.0,0.0,0.000000,0.000000,0.0,0.066667,0.000001,0.002569,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,0.00338,0.000000,0.0000,0.003671,0.001902,0.350507,0.000004,0.255172,0.010602,0.020501,0.032967,0.038462,0.0008,1.0000,0.0004,0.0022,0.083333,0.078125,0.046875,0.070423,0.03125,0.071429,0.085271,0.108696,0.087912,1.000000,1.000000,0.083333,0.016129,0.004188,0.007644,0.012501,0.003807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.213365,0.382166,0.000000,0.006682,0.000000,0.000000,0.244966,0.244966,0.207802,0.001200,0.001800,0.000840,0.001260,0.001200,0.001800,0.000840,0.001260,0.001119,0.001678,0.000783,0.001175,0.000009,0.000006,0.000013,0.000008,0.000009,0.000006,0.000013,0.000008,0.000028,0.000009,0.000040,0.000013,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16643.221785
1,0.240506,0.240506,0.250000,0.0,0.182243,0.185497,1.000000,0.666667,0.007273,1.0,0.00658,0.0052,0.0045,0.054945,0.032258,0.001814,0.197423,0.097701,0.0075,0.0,0.0,0.000000,0.000000,0.0,0.066667,0.004856,0.018869,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,0.00125,0.000000,0.0000,0.047174,0.014004,0.149493,0.000004,0.175172,0.015303,0.059226,0.095238,0.115385,0.0025,0.0075,0.0022,0.0075,0.083333,0.046875,0.046875,0.084507,0.02500,0.035714,0.069767,0.065217,0.054945,0.868317,0.504950,0.166667,0.016129,0.018550,0.004217,0.016449,0.005375,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.182884,0.212314,0.000000,0.007273,0.000000,0.000000,0.244966,0.244966,0.185736,0.001371,0.001372,0.001371,0.001372,0.001371

In [114]:
acc_subsample_nogrades.to_csv("acc_2_train_downsample_nogrades.csv.gz",
                              index=False,
                              compression="gzip")

# Viviann and Ali's Reduced Dataset

In [115]:
# Random Forest

acc_VC_rf = copy.deepcopy(acc_subsample[['int_rate', 'dti', 'avg_cur_bal', 'revol_bal',
                                         'duration_of_credit_months', 'ratio_inc_funded',
                                         'ratio_redinc_loan', 'installment', 'annual_inc',
                                         'loan_amnt', 'open_acc', 'grouped_subgrade_12_345',
                                         'emp_length', ]])

acc_VC_gb = copy.deepcopy(acc_subsample[['grouped_subgrade_12_345', 'grade', 'term',
                                         'ratio_inc_funded', 'max_bal_bc', 'int_rate', 'fico_score',
                                         'dti', 'emp_length', 'home_ownership', 'revol_bal',
                                         'annual_inc', 'duration_of_credit_months']])

acc_VC_rf.to_csv("acc_VC_rf.csv.gz",
                 index=False,
                 compression="gzip")

acc_VC_gb.to_csv("acc_VC_gb.csv.gz",
                 index=False,
                 compression="gzip")

# Theodore's Reduced Dataset

In [121]:
acc_TC = copy.deepcopy(acc_subsample[['annual_inc', 'annual_inc_joint', 'application_type', 'dti',
                                      'dti_joint', 'emp_length', 'duration_of_credit_months',
                                      'funded_amnt', 'fico_score', 'home_ownership', 'installment',
                                      'int_rate', 'loan_amnt', 'term', 'verification_status',
                                      'verification_status_joint', 'sec_app_fico_score',
                                      'sec_app_duration_of_credit_months', 'grade', 'sub_grade']])


acc_TC.to_csv("acc_TC.csv.gz",
              index=False,
              compression="gzip")

# Very Reduced Dataset

In [117]:
cols_w_purpose = [col for col in acc_subsample.columns if 'purpose' in col]
acc_subsample[cols_w_purpose]

,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
269355,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269356,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269357,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269358,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
cols_w_purpose = cols_w_purpose + ['annual_inc', 'fico_score', 'term', 'loan_amnt', 'home_ownership',
                                   'int_rate', 'all_util']

cols_w_purpose

['purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',
 'annual_inc',
 'fico_score',
 'term',
 'loan_amnt',
 'home_ownership',
 'int_rate',
 'all_util']

In [123]:
acc_vred = copy.deepcopy(acc_subsample[['annual_inc', 'fico_score', 'term', 'loan_amnt',
                                        'home_ownership', 'int_rate', 'all_util', 'dti',
                                        'total_pymnt']])


acc_vred.to_csv("acc_vred.csv.gz",
              index=False,
              compression="gzip")